In [2]:
import numpy as np
import matplotlib.pyplot as plt
from layers import *
from gradientCheck import *
from relativeError import *

In [2]:
np.random.seed(231)

# X = np.arange(60).reshape(3, 4, 5)

# W = np.random.randn(20, 5)

# b = np.random.rand(5)

# X = np.ones((2, 2)) * 10

# W = np.ones((2, 3)) * 2

# b = np.ones(3) * 3

x = np.random.randn(10, 2, 3)
w = np.random.randn(6, 5)
b = np.random.randn(5)
dout = np.random.randn(10, 5)

out, cache = affine_forward(X, W, b)

dout = 10 * np.random.randn(*out.shape)

dX, dW, db = affine_backward(dout, cache)

f_X = lambda x: affine_forward(x, W, b)[0]

f_W = lambda w: affine_forward(X, w, b)[0]

f_b = lambda b: affine_forward(X, W, b)[0]

# print(f_X(X).shape)
# print(X.reshape(3, -1).shape)

dX_num = numerical_gradient(f_X, X, dout)

dW_num = numerical_gradient(f_W, W, dout)

db_num = numerical_gradient(f_b, b, dout)

print('dX relative error: %f'%(relative_error(dX, dX_num)))

print(dX)

print(dX_num)

print('dW relative error: %f'%(relative_error(dW, dW_num)))

print('db relative error: %f'%(relative_error(db, db_num)))

dX relative error: 0.000000
[[  0.58278762   0.58278762]
 [-31.17139638 -31.17139638]]
[[  0.58278762   0.58278762]
 [-31.17139637 -31.17139637]]
dW relative error: 0.000000
db relative error: 0.000000


In [3]:
numerical_gradient(lambda x: x**2, np.arange(4).reshape(2, 2), np.ones((2, 2)))

[[0 1]
 [2 3]]
[[0 1]
 [2 3]]
[[0 0]
 [2 3]]
[[0 1]
 [2 3]]
[[0 1]
 [1 3]]
[[0 1]
 [2 3]]
[[0 1]
 [2 2]]
[[0 1]
 [2 3]]


array([[     0,  49999],
       [150000, 249999]])

In [4]:
numerical_gradient(lambda x: x**2, np.array([[2.,3.], [4.,5.]]), np.ones((2, 2)))


[[  3.99996   9.     ]
 [ 16.       25.     ]]
[[  4.00004   9.     ]
 [ 16.       25.     ]]
[[  4.        8.99994]
 [ 16.       25.     ]]
[[  4.        9.00006]
 [ 16.       25.     ]]
[[  4.        9.     ]
 [ 15.99992  25.     ]]
[[  4.        9.     ]
 [ 16.00008  25.     ]]
[[  4.       9.    ]
 [ 16.      24.9999]]
[[  4.       9.    ]
 [ 16.      25.0001]]


array([[ 4.,  6.],
       [ 8., 10.]])

In [19]:
arr = np.arange(4).reshape(2, 2)


In [20]:
arr


array([[0, 1],
       [2, 3]])

In [21]:
arr[1, 1]

3

In [22]:
arr[1,1] += 1e-5

In [23]:
arr


array([[0, 1],
       [2, 3]])

In [24]:
arr[1,1] -= 1e-5

In [25]:
arr

array([[0, 1],
       [2, 2]])